In [2]:
import pickle
import numpy as np
import copy

In [54]:
# read my pickle
# data = pickle.load(open("individuals_friendships_utilities.p","rb"))
# characteristics, connectivity, utilities = data

g_list = pickle.load(open("g_list.pkl", "rb"))

In [5]:
small = np.array([[0,1,1,0],
                 [1,0,1,0],
                 [0,0,0,0],
                 [0,0,0,0]])
print(small)
print(np.sum(small))

[[0 1 1 0]
 [1 0 1 0]
 [0 0 0 0]
 [0 0 0 0]]
4


In [12]:
# output:
# den_max_reach / density at maximum reach (RCHDEN): density of the reachability network 
# rel_den / relative density (RELDEN): amount of nodes divided by the maximum possible number of nodes
# p_symm_dyads / proportion symmetric dyads (PTCMUT): proportion of nodes that are reciprocal (counting once)
# mutuality_index / mutuality index (RHO2): Katz and Powell’s (1955) mutuality index. Measures the tendency for actors in a group to reciprocate choices.
den_max_reach, rel_den, p_symm_dyads, mutuality_index = analyse_network(small)

In [15]:
def analyse_network(connectivity):
    # Output:
    # density at maximum reach (RCHDEN), > den_max_reach
    # relative density (RELDEN), > rel_den
    # proportion symmetric dyads (PTCMUT), > p_symm_dyads 
    # mutuality index (RHO2), > mutuality_index
    
    # input: 
    # connectivity matrix with students claiming to have friends in row and students claimed to be befriended in columns
    
    nodes = connectivity.shape[0]
    mutual_d = 0
    asym_d = 0
    out_degrees = []
    
    
    #density DENX2
    #DENX2 = np.sum(connectivity)/(nodes*(nodes-1))
    
    
    # density at maximum reach RCHDEN
    
    # define the function to tranfer adjacency matrix to reachability matrix  
    # Prints reachability matrix of graph[][] using Floyd Warshall algorithm 
    # function found on https://www.geeksforgeeks.org/transitive-closure-of-a-graph/
    reachability = copy.deepcopy(connectivity)
    '''reach[][] will be the output matrix that will finally 
    have reachability values. 
    Initialize the solution matrix same as input graph matrix'''
    reach =[i[:] for i in reachability] 
    '''Add all vertices one by one to the set of intermediate 
    vertices. 
    ---> Before start of a iteration, we have reachability value 
    for all pairs of vertices such that the reachability values 
    consider only the vertices in set  
    {0, 1, 2, .. k-1} as intermediate vertices. 
    ----> After the end of an iteration, vertex no. k is 
    added to the set of intermediate vertices and the  
    set becomes {0, 1, 2, .. k}'''
    for k in range(nodes): 
              
        # Pick all vertices as source one by one 
        for i in range(nodes): 
                  
            # Pick all vertices as destination for the 
            # above picked source 
            for j in range(nodes): 
                      
                # If vertex k is on a path from i to j,  
                    # then make sure that the value of reach[i][j] is 1 
                reach[i][j] = reach[i][j] or (reach[i][k] and reach[k][j]) 
    
    RCHDEN = np.sum(reach)/(nodes*(nodes-1))
    
    
    # relative density RELDEN
    RELDEN = np.sum(connectivity)/(10*nodes)


    # create upper triangular matrix with 2's on mutual dyads, 1's on asymmetric dyads and count occurrence
    added_up = np.triu(connectivity + np.transpose(connectivity))
    mutual_d = np.count_nonzero(added_up == 2)
    asym_d = np.count_nonzero(added_up == 1)
    total_d = mutual_d + asym_d
    
    # calculate proportion symmetric dyads (PTCMUT) and asymmetric dyads (PTCASY)
    PTCMUT = mutual_d / total_d
    #PTCASY = asym_d / total_d
    
    
    # count total out_degree connections
    out_degree = connectivity.sum()
    # take the sum of squares of the out degree connections per individual (row)
    sum_squares_out = (connectivity.sum(axis=1)**2).sum()
    
    # calculate mutuality index (RHO2) (according to Katz and Powell’s (1955))
    RHO2 = (2*(nodes - 1)**2 * mutual_d - out_degree**2 + sum_squares_out) / (out_degree*(nodes - 1)**2 - out_degree**2 + sum_squares_out)

    
    return RCHDEN, RELDEN, PTCMUT, RHO2
    